<a href="https://colab.research.google.com/github/Sowmya-726/Trade-Sentiment-Analysis/blob/main/trade_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
os.listdir()


load the data

In [ ]:
import pandas as pd

trade_df = pd.read_csv('historical_data.csv')
sentiment_df = pd.read_csv('fear_greed_index.csv')

print(trade_df.head())

print(sentiment_df.head())

checking data structure


In [ ]:
# Trade
print("🔹 Trader Data Info:")
trade_df.info()
print("\n🔹 Null Values in Trade Data:")
print(trade_df.isnull().sum())

# Sentiment
print("\n🔹 Sentiment Data Info:")
sentiment_df.info()
print("\n🔹 Null Values in Sentiment Data:")
print(sentiment_df.isnull().sum())



In [ ]:
import pandas as pd

# 1. Converting 'Timestamp IST' in trade data to datetime
trade_df['Timestamp IST'] = pd.to_datetime(trade_df['Timestamp IST'], errors='coerce')
trade_df['date_only'] = trade_df['Timestamp IST'].dt.date

# 2. Convert 'date' in sentiment data to datetime and extract date
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'], errors='coerce')
sentiment_df['date_only'] = sentiment_df['date'].dt.date

# 3. Merge the datasets on the date column
merged_df = pd.merge(trade_df, sentiment_df, on='date_only', how='inner')

# 4. Preview the merged data
display(merged_df.head())

In [ ]:
print("historical",trade_df.shape)
print("sentiment",sentiment_df.shape)

In [ ]:
merged_df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(merged_df.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Data Heatmap")
plt.show()

In [ ]:
merged_df.describe()

In [ ]:
merged_df['classification'].value_counts()

In [ ]:
merged_df.groupby("classification")['Closed PnL'].mean().sort_values()


In [ ]:
merged_df['is_profitable'] = merged_df['Closed PnL'] > 0
win_rates = merged_df.groupby('classification')['is_profitable'].mean() * 100
print(win_rates)


In [ ]:
sns.boxplot(data=merged_df, x='classification', y='Closed PnL')
plt.title("Closed PnL by Sentiment")
plt.xticks(rotation=45)
plt.show()

In [ ]:
merged_df['is_profitable'] = merged_df['Closed PnL'] > 0


In [ ]:
summary = merged_df.groupby('classification').agg(
    avg_profit = ('Closed PnL', 'mean'),
    median_profit = ('Closed PnL', 'median'),
    profit_std = ('Closed PnL', 'std'),
    min_profit = ('Closed PnL', 'min'),
    max_profit = ('Closed PnL', 'max'),
    avg_trade_size = ('Size USD', 'mean'),
    median_trade_size = ('Size USD', 'median'),
    win_rate = ('is_profitable', 'mean')
).reset_index()

summary['win_rate'] = summary['win_rate'] * 100
summary = summary.round(2)
print(summary)



In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=summary, x='classification', y='avg_profit', order=summary['classification'])
plt.title('Average Profit/Loss by Market Sentiment')
plt.ylabel('Average Closed PnL')
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=summary, x='classification', y='win_rate', order=summary['classification'])
plt.title('Win Rate (%) by Market Sentiment')
plt.ylabel('Win Rate (%)')
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=merged_df, x='Side', hue='classification')
plt.title('Buy vs Sell Trade Counts by Market Sentiment')
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=merged_df, x='Closed PnL', hue='classification', bins=60, kde=True, element='step')
plt.title('Profit/Loss Distribution by Market Sentiment')
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=merged_df, x='Size USD', y='Closed PnL', hue='classification', alpha=0.6)
plt.title('Trade Size vs Profit by Sentiment')
plt.show()


In [ ]:
trade_counts = merged_df['classification'].value_counts()
print("Number of trades per sentiment:")
print(trade_counts)


In [ ]:
print("Win rate (% profitable trades) by sentiment:")
print(summary[['classification', 'win_rate']])